In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!!sudo apt-get install swig3.0

In [ ]:
!pip install nltk
!pip install seaborn
!pip install datasets transformers
!pip install tensorflow_text
from keras.utils import pad_sequences
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score ,confusion_matrix
import pandas as pd
import numpy as np
import transformers
from transformers import BertTokenizer
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from gensim.utils import simple_preprocess
import tensorflow_text as text
import re
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from keras.layers import Input, Embedding, Dense, Bidirectional, Dropout, GRU
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.layers import SpatialDropout1D, Conv1D, MaxPooling1D, LSTM

****Loading the Data****

In [ ]:
dataset_path = "/kaggle/input/train-dataset-csv/train_dataset.csv"
test_path = "/kaggle/input/test-data-set-csv/test_dataset.csv"
sample_submission = pd.read_csv("/kaggle/input/sample-submission/sample_submission.csv")
train_data = pd.read_csv(dataset_path)
test_data = pd.read_csv(test_path)
train_data.drop(columns=['qid'], inplace=True)

In [ ]:
train_data.head()

**Pre Processing**

Remove stopwords and preprocessing.

In [ ]:
def  clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"'ll", " will", text)
    text = re.sub(r"'ve", " have", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"'d", " would", text)
    text = re.sub(r"'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()#/@;:{}`+=~|.!?,]", "", text)
    text = text.translate(str.maketrans('', '', string.punctuation)) 
    text = re.sub("(\W)"," ",text) 
    return text

def adjust(sen):
    return trie.getWord(sen)

def preprocess(train_data):
    train_data['question_text'] = train_data['question_text'].apply(lambda x: clean_text(x))
    train_data['question_text'] = train_data['question_text'].apply(lambda x: adjust(x))
    train_data['question_text'] = train_data['question_text'].apply(lambda x: simple_preprocess(x,max_len=30))
    train_data['question_text'] = train_data['question_text'].apply(lambda x: [w for w in x if w not in english_stopwords])

**Using pretrained Glove vectors to build the dictionary of words to vectors**

In [ ]:
embeddings_index = {}
vocabulary = []
f= open('/kaggle/input/glove-200d/glove.6B.200d.txt')
for line in f:
    values = line.split()
    word = values[0]
    vocabulary.append(word)
    coeffs = np.asarray(values[1:],dtype='float16')
    embeddings_index[word] = coeffs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

**Building a trie to handle out of vocabulary problem**

In [ ]:
class Trie:
    def __init__(self,char,children,isend=False):
        self.char = char
        self.children = children
        self.isend = isend
    
    def insert(self, word):
        node = self
        for char in word:
            node = node.children.setdefault(char,Trie(char,{}))
        node.isend = True
    
      
    
    def isSeqPresent(self,word):
        node = self
        for char in word:
            node = node.children.get(char,None)
            if not node:
                return 0
        return 1
    
    def getWord(self, word, n=2):
        node = self
        result = ""
        letter_count = 0
        sug = []
        ans=''
        for char in word:
            if char == ' ':
                node = self
                ans += result + ' '
                result = ""
                letter_count = 0
                sug = []
                continue
            if not node.children.get(char,None):
                if len(sug) > 0:
                    val = sug.pop()
            # carry the excess chars from previous word
                    result = result.replace(val,'')
                    ans += val+' '
                else: result = ''
                node = self
                letter_count =len(result)
            if node.children.get(char,None):
                node = node.children[char]
                result +=char
                letter_count +=1
    #          for two letter words and above  maintain a stack
                if letter_count>1 and node.isend:
                    sug.append(result)
        ans += result
        return ans

# class Info:
#     def __init__(self, result='',ans='',sug):
#         self.result = result
#         self.ans = ans
#         self.sug = sug
        
def get_word_2(self, word, n=2):
        node = self
        result = ""
        letter_count = 0
        sug = []
        ans=''
        for char in word:
            if char == ' ':
                node = self
                ans += result + ' '
                result = ""
                letter_count = 0
                sug = []
                continue
            if not node.children.get(char,None):
                if len(sug) > 0:
                    val = sug.pop()
                    sug = []
            # carry the excess chars from previous word
                    result = result.replace(val,'')
                    ans += val+' '
                else: result = ''
                node = self
                for i in result:
                    if node.children.get(i,None):
                        node = node.children[i]
                    else: 
                        node = self
                        result = ''
                letter_count =len(result)
            if node.children.get(char,None):
                node = node.children[char]
                result +=char
                letter_count +=1
        #          for two letter words and above  maintain a stack
                if letter_count>1 and node.isend:
                    sug.append(result)
        ans += result
        return ans

print('initialising Trie')

trie = Trie('dummy',{})
def build_trie(vocabulary_trim):
    for word in vocabulary_trim:
        trie.insert(word)
        
trie.getWord = get_word_2.__get__(trie)
print('initialising vocabulary')
vocabulary_trim = [x for x in vocabulary if len(x)>1]
print('The length of trimmed vocabulary',len(vocabulary_trim))
print('building trie')
build_trie(vocabulary_trim)
print('successfully built the trie')

**Testing the trie**

In [ ]:
# sample words that trie effectively handles

trie.getWord('exboyfriend')
trie.isPresent('demonetize')
trie.getWord('bestfriend, nonmuslim, dropshipping quorans pakistangs womangs persongs himher exgirlfriend altright')

# Preprocessing 

In [ ]:
preprocess(train_data)
preprocess(test_data)
test_data.head()

y=train_data['target'].values
print(y[:5])

# Compute class weights
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y)
class_weights = dict(enumerate(class_weights))

In [ ]:
def target_ratio(y):
    count = 0
    for val in y:
        if val==1: count+=1
    print('number 1s are %s and number of 0s are %s and ratio is %s'%(count, len(y)-count, count/len(y)))

# verifying the class weights
print('class weights', class_weights)
target_ratio(y)


# Tokenizing the text

In [ ]:
MAX_LEN=30
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['question_text'])
X=tokenizer.texts_to_sequences([' '.join(seq[:MAX_LEN]) for seq in train_data['question_text']])
X_test_data=tokenizer.texts_to_sequences([' '.join(seq[:MAX_LEN]) for seq in test_data['question_text']])

# padding the train data
X = pad_sequences(X, maxlen=MAX_LEN, padding='post', truncating='post')

# padding the test data
X_test_data = pad_sequences(X_test_data, maxlen=MAX_LEN, padding='post', truncating='post')

# splitting the train data into train and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)

# Building the embedding matrix

In [ ]:
words_not_found = []
vocab_size = len(tokenizer.word_index)+1
embedding_dim = 200
embedding_matrix = np.zeros((vocab_size,embedding_dim))
for word,i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)

**Identifying the OOV**

In [ ]:
test_vocab = words_not_found[:100]
test_vocab = ' '.join(test_vocab)
test_vocab
words_cs = pd.DataFrame(words_not_found)
words_cs.to_csv('words_not_found.csv', index=False)

# Building Model

In [ ]:
model = Sequential()
embedding_layer = Embedding(vocab_size,
                            embedding_dim,
                            weights = [embedding_matrix],
                            input_length = MAX_LEN,
                            trainable=False)
model.add(embedding_layer)
model.add(Bidirectional(GRU(128, return_sequences=True, dropout=0.50, name='first_gru_layer')))
model.add(Dropout(0.5))
model.add(Bidirectional(GRU(64, name='second_gru_layer')))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid', name='output_layer'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
BATCH_SIZE = 128
N_EPOCHS = 8

In [ ]:
model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=N_EPOCHS,
          validation_data=(X_test, y_test),
          class_weight=class_weights)

# Predicting with the model

In [ ]:
# obtaining accuracy with the validation data
y_s = model.predict(X_test)
y_s_r = [1 if val > 0.83 else 0 for val in y_s]
y_s_r[:2]



cr = pd.DataFrame(classification_report(y_test,y_s_r,output_dict=True)).T
cr['support'] = cr.support.apply(int)
cr.style.background_gradient(cmap='Pastel1')

In [ ]:
# Getting the predictions on the test data
y_test_data = model.predict(X_test_data)
y_test_data1[:2]
y_test_data1 = [1 if val > 0.8 else 0 for val in y_test_data]
test_data['target'] = y_test_data1
test_data['target'].value_counts()
test_data.drop(columns=["question_text"],inplace=True)
test_data['qid'] = sample_submission['qid']
test_data.to_csv("test_results.csv", index=False)